In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree_models').getOrCreate()

In [2]:
data = spark.read.csv("D:\\Udemy course\\Spark\\MLLib\\College.csv", inferSchema=True, header=True)
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [5]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [6]:
#setting up predictors into a vector....
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'], outputCol='features')

In [7]:
#transforming the data into vector....
output = assembler.transform(data)

In [8]:
output.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60, features=DenseVector([1660.0, 1232.0, 721.0, 23.0, 52.0, 2885.0, 537.0, 7440.0, 3300.0, 450.0, 2200.0, 70.0, 78.0, 18.1, 12.0, 7041.0, 60.0]))]

In [11]:
#converting the categorical target variable "Private" into numeric....
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='Private',outputCol='privateindex')
new_df = indexer.fit(output).transform(output)

In [12]:
new_df.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60, features=DenseVector([1660.0, 1232.0, 721.0, 23.0, 52.0, 2885.0, 537.0, 7440.0, 3300.0, 450.0, 2200.0, 70.0, 78.0, 18.1, 12.0, 7041.0, 60.0]), privateindex=0.0)]

In [13]:
#final predictors and target variable....
final_data = new_df.select(['features','privateindex'])
final_data.show()

+--------------------+------------+
|            features|privateindex|
+--------------------+------------+
|[1660.0,1232.0,72...|         0.0|
|[2186.0,1924.0,51...|         0.0|
|[1428.0,1097.0,33...|         0.0|
|[417.0,349.0,137....|         0.0|
|[193.0,146.0,55.0...|         0.0|
|[587.0,479.0,158....|         0.0|
|[353.0,340.0,103....|         0.0|
|[1899.0,1720.0,48...|         0.0|
|[1038.0,839.0,227...|         0.0|
|[582.0,498.0,172....|         0.0|
|[1732.0,1425.0,47...|         0.0|
|[2652.0,1900.0,48...|         0.0|
|[1179.0,780.0,290...|         0.0|
|[1267.0,1080.0,38...|         0.0|
|[494.0,313.0,157....|         0.0|
|[1420.0,1093.0,22...|         0.0|
|[4302.0,992.0,418...|         0.0|
|[1216.0,908.0,423...|         0.0|
|[1130.0,704.0,322...|         0.0|
|[3540.0,2001.0,10...|         1.0|
+--------------------+------------+
only showing top 20 rows



In [14]:
#split data
train,test = final_data.randomSplit([0.8,0.2])

In [16]:
#initializing our models...
from pyspark.ml.classification import (DecisionTreeClassifier, RandomForestClassifier, GBTClassifier)
dtc = DecisionTreeClassifier(featuresCol='features', labelCol='privateindex')
rfc = RandomForestClassifier(featuresCol='features', labelCol='privateindex', numTrees=100)
gbt = GBTClassifier(featuresCol='features', labelCol='privateindex')

In [17]:
#t
dtcc = dtc.fit(train)
rfcc = rfc.fit(train)
gbtt = gbt.fit(train)

In [18]:
dtc_pred = dtcc.transform(test)
rfc_pred = rfcc.transform(test)
gbt_pred = gbtt.transform(test)

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluation = BinaryClassificationEvaluator(labelCol='privateindex')

In [22]:
print("Decision tree accuracy: {0}".format(evaluation.evaluate(dtc_pred)))
print("Random forest accuracy: {0}".format(evaluation.evaluate(rfc_pred)))

evaluation2 = BinaryClassificationEvaluator(labelCol='privateindex', rawPredictionCol='prediction')
print("Gradient boosted accuracy: {0}".format(evaluation2.evaluate(gbt_pred)))

Decision tree accuracy: 0.9380942538238942
Random forest accuracy: 0.9780901198842497
Gradient boosted accuracy: 0.9056428276147167
